In [15]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy

In [16]:
d = 2

In [17]:
def product_state(ttn, bond_dim=2 , physical_dim= 2):
    product_state = deepcopy(ttn)
    A = np.array([1, 0]) 
    #A = np.random.rand(2) + 1j * np.random.rand(2)
    for node_id in product_state.nodes.keys():
        n = product_state.tensors[node_id].ndim - 1
        tensor = A.reshape((1,) * n + (physical_dim,))
        T = np.pad(tensor, n*((bond_dim-1, bond_dim-1),) + ((0, 0),))
        product_state.tensors[node_id] = T
        product_state.nodes[node_id].link_tensor(T)  
    return product_state

In [18]:
### (0, 0): (2, 2, 2) --> (2, 2, 2, 2)

shapes = {
    (0, 0): (2, 2, 2, 2),
    (0, 1): (2, 2, 2, 2),
    (0, 2): (2, 2, 2, 2),
    (0, 3): (2, 2),
    (1, 0): (2, 2, 2),
    (1, 1): (2, 2),
    (1, 2): (2, 2, 2),
    (1, 3): (2, 2),
    (2, 0): (2, 2, 2, 2),
    (2, 1): (2, 2, 2),
    (2, 2): (2, 2, 2),
    (2, 3): (2, 2),
    (3, 0): (2, 2),
    (3, 1): (2, 2, 2),
    (3, 2): (2, 2, 2, 2),
    (3, 3): (2, 2)
}

sites = {
    (i, j): ptn.random_tensor_node(shapes[(i, j)], identifier=f"Site({i},{j})") for i in range(4) for j in range(4)
}

ttn = ptn.TreeTensorNetworkState()

ttn.add_root(sites[(0, 0)][0], sites[(0, 0)][1])

connections = [
    ((0, 0), (0, 1), 0, 0),
    ((0, 1), (1, 1), 1, 0),
    ((0, 1), (0, 2), 2, 0),
    ((0, 2), (1, 2), 1, 0),
    ((1, 2), (1, 3), 1, 0),
    ((0, 2), (0, 3), 2, 0),
    ((0, 0), (1, 0), 1, 0),
    ((1, 0), (2, 0), 1, 0),
    ((2, 0), (3, 0), 1, 0),
    ((2, 0), (2, 1), 2, 0),
    ((2, 1), (3, 1), 1, 0),
    ((3, 1), (3, 2), 1, 0),
    ((3, 2), (2, 2), 1, 0),
    ((2, 2), (2, 3), 1, 0),
    ((3, 2), (3, 3), 2, 0),
]

for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Site({parent[0]},{parent[1]})"
    child_id = f"Site({child[0]},{child[1]})"
    ttn.add_child_to_parent(sites[child][0], sites[child][1], child_leg, parent_id, parent_leg)

ttn = product_state(ttn , bond_dim= 4, physical_dim = 2)  

nodes = {
    (i, j): (ptn.Node(tensor=ttn.tensors[f"Site({i},{j})"].conj() , identifier=f"Node({i},{j})"), ttn.tensors[f"Site({i},{j})"].conj()) for i in range(4) for j in range(4)
}

ttn.add_child_to_parent(nodes[(0,0)][0], nodes[(0,0)][1], 2, "Site(0,0)", 2)

### (0,0) : 0,1 --> 1,2

connections = [
    ((0, 0), (0, 1), 1, 0),
    ((0, 1), (1, 1), 1, 0),
    ((0, 1), (0, 2), 2, 0),
    ((0, 2), (1, 2), 1, 0),
    ((1, 2), (1, 3), 1, 0),
    ((0, 2), (0, 3), 2, 0),
    ((0, 0), (1, 0), 2, 0),
    ((1, 0), (2, 0), 1, 0),
    ((2, 0), (3, 0), 1, 0),
    ((2, 0), (2, 1), 2, 0),
    ((2, 1), (3, 1), 1, 0),
    ((3, 1), (3, 2), 1, 0),
    ((3, 2), (2, 2), 1, 0),
    ((2, 2), (2, 3), 1, 0),
    ((3, 2), (3, 3), 2, 0),
]

for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Node({parent[0]},{parent[1]})"
    child_id = f"Node({child[0]},{child[1]})"
    ttn.add_child_to_parent(nodes[child][0], nodes[child][1], child_leg, parent_id, parent_leg)

In [19]:
shapes = {
    (0, 0): (3, 5, 6, d),
    (0, 1): (3, 7, d),
    (0, 2): (7, 8, d),
    (1, 0): (5, 5, d),
    (1, 1): (9, d),
    (1, 2): (8, d),
    (2, 0): (5, 6, d),
    (2, 1): (6, 9, 3, d),
    (2, 2): (3, d)
}


sites = {
    (i, j): ptn.random_tensor_node(shapes[(i, j)], identifier=f"Site({i},{j})") for i in range(3) for j in range(3)
}

ttn = ptn.TreeTensorNetworkState()

ttn.add_root(sites[(0, 0)][0], sites[(0, 0)][1])

connections = [
    ((0, 0), (0, 1), 0, 0),
    ((0, 1), (0, 2), 1, 0),
    ((0, 2), (1, 2), 1, 0),
    ((0, 0), (1, 0), 1, 0),
    ((1, 0), (2, 0), 1, 0),
    ((2, 0), (2, 1), 1, 0),
    ((2, 1), (1, 1), 1, 0),
    ((2, 1), (2, 2), 2, 0)]


for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Site({parent[0]},{parent[1]})"
    child_id = f"Site({child[0]},{child[1]})"
    ttn.add_child_to_parent(sites[child][0], sites[child][1], child_leg, parent_id, parent_leg)

ttn = product_state(ttn , bond_dim= 4, physical_dim = d)

nodes = {
    (i, j): (ptn.Node(tensor=ttn.tensors[f"Site({i},{j})"] , identifier=f"Node({i},{j})"), ttn.tensors[f"Site({i},{j})"]) for i in range(3) for j in range(3)
}

ttn.add_child_to_parent(nodes[(0,0)][0], nodes[(0,0)][1], 2, "Site(0,0)", 2)

connections = [
    ((0, 0), (0, 1), 1, 0),
    ((0, 1), (0, 2), 1, 0),
    ((0, 2), (1, 2), 1, 0),
    ((0, 0), (1, 0), 2, 0),
    ((1, 0), (2, 0), 1, 0),
    ((2, 0), (2, 1), 1, 0),
    ((2, 1), (1, 1), 1, 0),
    ((2, 1), (2, 2), 2, 0),
]

for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Node({parent[0]},{parent[1]})"
    child_id = f"Node({child[0]},{child[1]})"
    ttn.add_child_to_parent(nodes[child][0], nodes[child][1], child_leg, parent_id, parent_leg)

In [20]:
def get_neighbors_periodic_in_legs(x, y, Lx, Ly):
  neighbors = []
  
  # Right neighbor (with periodic boundary)
  right_x = (x + 1) % Lx
  neighbors.append((f"Site({right_x},{y})"))
  
  # Up neighbor (with periodic boundary)
  up_y = (y + 1) % Ly
  neighbors.append((f"Site({x},{up_y})"))
  
  return neighbors

def get_neighbors_periodic_out_legs(x, y, Lx, Ly):
    neighbors = []
    
    # Right neighbor (with periodic boundary)
    right_x = (x + 1) % Lx
    neighbors.append((f"Node({right_x},{y})"))
    
    # Up neighbor (with periodic boundary)
    up_y = (y + 1) % Ly
    neighbors.append((f"Node({x},{up_y})"))
    
    return neighbors  



In [21]:
def Anisotropic_Heisenberg_ham(J_x, J_y, J_z, h_z, Lx, Ly):
    # Get the Pauli matrices
    X, Y, Z = ptn.pauli_matrices()
    
    # Create a conversion dictionary for the operators
    conversion_dict = {
        "X": X,
        "J_x * X": J_x * X,
        "Y": Y,
        "J_y * Y": J_y * Y,
        "Z": Z,
        "J_z * Z": J_z * Z,
        "I2": np.eye(2),
        "h_z * Z": h_z * Z
    }
    
    terms = []
    
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            neighbors = get_neighbors_periodic_in_legs(x, y, Lx, Ly)
            
            for neighbor in neighbors:
                terms.append(ptn.TensorProduct({current_site: "X", neighbor: "J_x * X"}))
                terms.append(ptn.TensorProduct({current_site: "Y", neighbor: "J_y * Y"}))
                terms.append(ptn.TensorProduct({current_site: "Z", neighbor: "J_z * Z"}))               

    
    # On-site magnetic field terms
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            terms.append(ptn.TensorProduct({current_site: "h_z * Z"}))
    
    return ptn.Hamiltonian(terms, conversion_dict)

In [22]:
def Liouville_Heisenberg(Lx, Ly, J_x, J_y, J_z, h_z, L, J):
    # Get the Pauli matrices
    X, Y, Z = ptn.pauli_matrices()
    
    # Create the conversion dictionary for the Hamiltonian.
    conversion_dict = {
        "-iJ_x * X": -1j*J_x * (X),
        "X":                   (X),
        "-iJ_Y * Y": -1j*J_y * (Y),
        "Y":                   (Y),
        "-iJ_z * Z": -1j*J_z * (Z),
        "Z":                   (Z),
        "I2":          np.eye(2),
        "-ih_z * Z":  -1j* h_z * Z
    }
    
    terms = []
    
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            neighbors = get_neighbors_periodic_in_legs(x, y, Lx, Ly)
            
            for neighbor in neighbors:
                terms.append(ptn.TensorProduct({current_site: "-iJ_x * X", neighbor: "X"}))
                terms.append(ptn.TensorProduct({current_site: "-iJ_Y * Y", neighbor: "Y"}))
                terms.append(ptn.TensorProduct({current_site: "-iJ_z * Z", neighbor: "Z"}))     

    
    # On-site magnetic field terms
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            terms.append(ptn.TensorProduct({current_site: "-ih_z * Z"}))

    H1 = ptn.Hamiltonian(terms, conversion_dict)
    
    # H.T = H 
    
    # Create the conversion dictionary for the Hamiltonian
    conversion_dict = {
        "iJ_x * X.T": 1j*J_x * (X.T),
        "X.T":                 (X.T),
        "iJ_Y * Y.T": 1j*J_y * (Y.T),
        "Y.T":                 (Y.T),
        "iJ_z * Z.T": 1j*J_z * (Z.T),
        "Z.T":                 (Z.T),
        "I2":              np.eye(2),
        "ih_z * Z.T":     1j* h_z * (Z.T)
    }
    
    terms = []
    
    # Hopping terms for the transpose
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Node({x},{y})"
            neighbors = get_neighbors_periodic_out_legs(x, y, Lx, Ly)
            
            for neighbor in neighbors:
                terms.append(ptn.TensorProduct({current_site: "iJ_x * X.T", neighbor: "X.T"}))
                terms.append(ptn.TensorProduct({current_site: "iJ_Y * Y.T", neighbor: "Y.T"}))
                terms.append(ptn.TensorProduct({current_site: "iJ_z * Z.T", neighbor: "Z.T"})) 


    # On-site magnetic field terms
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Node({x},{y})"
            terms.append(ptn.TensorProduct({current_site: "ih_z * Z.T"}))
    
    H2 = ptn.Hamiltonian(terms, conversion_dict)    
    H1.__add__(H2)
    
    conversion_dict = {
        "L": np.sqrt(J) * L,
        "L^dagger.T": np.sqrt(J) * L.conj().T,
        "-1/2 (L^dagger @ L)": -1/2 * J * (L.conj().T @ L),
        "-1/2 (L^dagger @ L).T": -1/2 * J * (L.conj().T @ L).T
    }

    terms = []
    for x in range(Lx):
        for y in range(Ly):
            out_site = f"Node({x},{y})"
            in_site = f"Site({x},{y})"
            terms.append(ptn.TensorProduct({in_site: "L"}))
            terms.append(ptn.TensorProduct({out_site: "L^dagger.T"}))
            terms.append(ptn.TensorProduct({in_site: "-1/2 (L^dagger @ L)"}))
            terms.append(ptn.TensorProduct({out_site: "-1/2 (L^dagger @ L).T"}))

    H3 = ptn.Hamiltonian(terms, conversion_dict)
    H1.__add__(H3)
    
    return H1

In [23]:
def Unitary(Lx, Ly, J_x, J_y, J_z, h_z):
    # Get the Pauli matrices
    X, Y, Z = ptn.pauli_matrices()
    
    # Create the conversion dictionary for the Hamiltonian
    conversion_dict = {
        "-i * X": -1j* X,
        "J_x * X": J_x * X,
        "-i * Y": -1j* Y,
        "J_y * Y": J_y * Y,
        "-i * Z": -1j* Z,
        "J_z * Z": J_z * Z,
        "I2": np.eye(2),
        "-i * h_z * Z": -1j* h_z * Z
    }
    
    terms = []
    
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            neighbors = get_neighbors_periodic_in_legs(x, y, Lx, Ly)
            
            for neighbor in neighbors:
                terms.append(ptn.TensorProduct({current_site: "-i * X", neighbor: "J_x * X"}))
                terms.append(ptn.TensorProduct({current_site: "-i * Y", neighbor: "J_y * Y"}))
                terms.append(ptn.TensorProduct({current_site: "-i * Z", neighbor: "J_z * Z"}))       

    
    # On-site magnetic field terms
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            terms.append(ptn.TensorProduct({current_site: "-i * h_z * Z"}))

    H1 = ptn.Hamiltonian(terms, conversion_dict)
    
    # H.T = H 
    
    # Create the conversion dictionary for the Hamiltonian
    conversion_dict = {
        "i * X": 1j* X,
        "J_x * X": J_x * X,
        "i * Y": 1j* Y,
        "J_y * Y": J_y * Y,
        "i * Z": 1j* Z,
        "J_z * Z": J_z * Z,
        "I2": np.eye(2),
        "i * h_z * Z": 1j* h_z * Z
    }
    
    terms = []
    
    # Hopping terms for the transpose
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Node({x},{y})"
            neighbors = get_neighbors_periodic_out_legs(x, y, Lx, Ly)
            
            for neighbor in neighbors:
                terms.append(ptn.TensorProduct({current_site: "i * X", neighbor: "J_x * X"}))
                terms.append(ptn.TensorProduct({current_site: "i * Y", neighbor: "J_y * Y"}))
                terms.append(ptn.TensorProduct({current_site: "i * Z", neighbor: "J_z * Z"})) 


    # On-site magnetic field terms
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Node({x},{y})"
            terms.append(ptn.TensorProduct({current_site: "i * h_z * Z"}))
    
    H2 = ptn.Hamiltonian(terms, conversion_dict)    
    H1.__add__(H2)
    
    return H1    
    

In [24]:
def Magnetization_op_total(Lx, Ly):
    # Get the Pauli matrices
    X, Y, Z = ptn.pauli_matrices()
    
    # Create a conversion dictionary for the operators
    conversion_dict = {
        "X": X ,
        "Y": Y,
        "Z": Z,
        "I2": np.eye(2)}
    terms = []
    for x in range(Lx):
        for y in range(Ly):
            current_site = f"Site({x},{y})"
            terms.append(ptn.TensorProduct({current_site: "Z"}))  # Using Z for magnetization

    return ptn.Hamiltonian(terms, conversion_dict)

In [25]:
Lx =  3
Ly  = 3
J_x = 1
J_y = 1.5 #(1.8)
J_z = 0.4 
h_z = 0


# TTNO : Hamiltonian acting on in_legs
H1 = Anisotropic_Heisenberg_ham(J_x, J_y, J_z, h_z, Lx, Ly)
H1 = H1.pad_with_identities(ttn, symbolic= True)
H = ptn.TTNO.from_hamiltonian(H1, ttn)

# TTNO : Liouville operator 
X , Y , Z = ptn.pauli_matrices()
L = (X - 1j * Y) / 2
J = 0.2
H1 = Liouville_Heisenberg(Lx, Ly, J_x, J_y, J_z, h_z, L, J)
H1 = H1.pad_with_identities(ttn , symbolic= True)
L_fancy = ptn.TTNO.from_hamiltonian(H1, ttn)

# TTNO : Unitary operator
H1 = Unitary(Lx, Ly, J_x, J_y, J_z, h_z)
H1 = H1.pad_with_identities(ttn, symbolic= True)
U = ptn.TTNO.from_hamiltonian(H1, ttn)

M = Magnetization_op_total(Lx, Ly)
M = M.pad_with_identities(ttn, symbolic= True)
M = ptn.TTNO.from_hamiltonian(M, ttn)

#ttn = ptn.normalize_ttn_Lindblad_4(ttn , 'Node(2,3)')
I = ptn.TTNO.Identity(ttn)
print(ttn.operator_expectation_value_Lindblad(I))
print(ttn.operator_expectation_value_Lindblad(M) / ttn.operator_expectation_value_Lindblad(I))

(1+0j)
(9+0j)


In [26]:
ttn = ptn.normalize_ttn_Lindblad_1_conj(ttn)
tdvp_ex1 = ptn.SecondOrderOneSiteTDVP(initial_state = ttn,
                                     hamiltonian = L_fancy,
                                     time_step_size = 0.01,
                                     final_time = 2,
                                     operators = M,
                                     num_vecs = 3,
                                     tau = 1e-3,
                                     SVDParameters = ptn.SVDParameters(max_bond_dim = np.inf , rel_tol= -np.inf , total_tol = -np.inf),
                                     expansion_steps = 150000,
                                     t3n_dict= {'Site(0,0)': 'Site(1,0)',
                                                'Site(2,1)': 'Site(1,1)',
                                                'Node(0,0)': 'Node(1,0)',
                                                'Node(2,1)': 'Node(1,1)'},

                                     Lanczos_threshold = np.inf,
                                     k_fraction = 0.2, 
                                     validity_fraction = 1, 
                                     increase_fraction = 0.3,
                                     max_iter = 1, 

                                     initial_tol= 1e-16,
                                     tol_step= 10,
                                     rel_tot_bond = 30,
                                     max_bond= 220,
                                     KrylovBasisMode = ptn.KrylovBasisMode.apply_1st_order_expansion,
                                     config = ptn.TTNTimeEvolutionConfig(record_bond_dim=True,
                                                                         Lindblad = True,
                                                                         T3NS= False) )

In [27]:
stop

NameError: name 'stop' is not defined

In [28]:
tdvp_ex1.run_ex(evaluation_time=1)


  0%|          | 0/201 [00:00<?, ?it/s]

M : (9+0j)


  0%|          | 1/201 [00:00<01:39,  2.00it/s]

M : (9.001517144228425+1.1273033261868673e-05j)


  1%|          | 2/201 [00:00<01:09,  2.88it/s]

...

  4%|▍         | 8/201 [00:02<00:55,  3.45it/s]

M : (9.072217040672841-0.0011798082982750735j)


  4%|▍         | 9/201 [00:02<00:58,  3.31it/s]

M : (9.080203093434582-0.00422467897578762j)


KeyboardInterrupt: 

In [134]:
from qutip import *
import numpy as np

# Define coupling constants for anisotropy
Jx = 1  # Coupling in x-direction
Jy = 1.5 # Coupling in y-direction
Jz = 0.4  # Coupling in z-direction

gamma_relax = 0.2 # Relaxation rate
# gamma_dephase = 1  # Dephasing rate

# Lattice dimensions
Nx = 3  # Number of sites along x-direction
Ny = 3  # Number of sites along y-direction
N = Nx * Ny  # Total number of sites

# Precompute the spin operators for each site
sx_list = []
sy_list = []
sz_list = []
sigmam_list = []

si = qeye(2)  # Identity operator for a single qubit
for n in range(N):
  # List of identity operators for all sites
  op_list = [si] * N
  # Sx operator at site n
  op_list[n] = sigmax()
  sx = tensor(op_list)
  sx_list.append(sx)
  
  # Sy operator at site n
  op_list[n] = sigmay()
  sy = tensor(op_list)
  sy_list.append(sy)
  
  # Sz operator at site n
  op_list[n] = sigmaz()
  sz = tensor(op_list)
  sz_list.append(sz)
  
  # S- (lowering) operator at site n
  op_list[n] = sigmam()
  sigmam_op = tensor(op_list)
  sigmam_list.append(sigmam_op)

# Function to map 2D lattice coordinates (i, j) to a site index
def site(i, j):
  """Return the site index for coordinates (i, j), with periodic boundaries."""
  return (i % Nx) + (j % Ny) * Nx

# Initialize the Hamiltonian
H = 0

# Build the Hamiltonian by summing over nearest neighbors with periodic boundary conditions
for i in range(Nx):
  for j in range(Ny):
      n = site(i, j)  # Current site index
      # Right neighbor (periodic boundary)
      n_right = site(i + 1, j)
      # Interaction terms with the right neighbor
      H += Jx * sx_list[n] * sx_list[n_right]
      H += Jy * sy_list[n] * sy_list[n_right]
      H += Jz * sz_list[n] * sz_list[n_right]
      
      # Up neighbor (periodic boundary)
      n_up = site(i, j + 1)
      # Interaction terms with the up neighbor
      H += Jx * sx_list[n] * sx_list[n_up]
      H += Jy * sy_list[n] * sy_list[n_up]
      H += Jz * sz_list[n] * sz_list[n_up]
      

# Option 1: All spins up |000...0> (Eigenstate of Sz_total)
psi0 = tensor([basis(2, 0) for _ in range(N)]).unit()

# Option 2: All spins in superposition |+⟩ = (|0⟩ + |1⟩)/sqrt(2)
# psi0 = tensor([(basis(2, 0) + basis(2, 1)).unit() for _ in range(N)])

# Option 3: Half spins excited |111...1000...0>
# psi0 = tensor([basis(2, 1) if n < N//2 else basis(2, 0) for n in range(N)]).unit()


# Define collapse operators (for the Lindblad equation)
# For example, include relaxation and dephasing at each site


c_ops = []

for n in range(N):
  # a. Relaxation (spin flip down) using lowering operator S-
  c_ops.append(np.sqrt(gamma_relax) * sigmam_list[n])
  
  # b. Optional: Dephasing using Sz operator
  # Uncomment the following lines if you wish to include dephasing
  # c_ops.append(np.sqrt(gamma_dephase) * sz_list[n])

# Time points where we want the solution
total_time = 0.1  # Total time in seconds
time_step = 0.001  # Time step in seconds

# Create the time list
tlist = np.arange(0, total_time + time_step, time_step)  # Include the endpoint


# Observables to calculate - for instance, total magnetization in z-direction
sz_total = sum(sz_list)

# Solve the master equation
result = mesolve(
  H,        # Hamiltonian
  psi0,     # Initial state
  tlist,    # Time list
  c_ops,    # Collapse operators
  [sz_total]  # List of observables
  # , [sz_first, sz_last]  # Uncomment to track individual spins
)

# Extract expectation values
magnetization_z = result.expect[0]

In [135]:
magnetization_z

array([9.        , 8.99638237, 8.99272948, 8.98904139, 8.98531812,
       8.98155972, 8.97776622, 8.9739377 , 8.97007419, 8.96617575,
       8.96224243, 8.95827431, 8.95427148, 8.950234  , 8.94616193,
       8.94205534, 8.9379143 , 8.93373888, 8.92952923, 8.92528544,
       8.92100761, 8.91669581, 8.91235014, 8.90797069, 8.90355755,
       8.89911088, 8.89463079, 8.8901174 , 8.88557084, 8.88099124,
       8.87637873, 8.87173345, 8.86705552, 8.86234509, 8.85760232,
       8.85282734, 8.84802031, 8.84318138, 8.83831071, 8.83340845,
       8.82847477, 8.82350982, 8.81851379, 8.81348684, 8.80842915,
       8.80334091, 8.79822228, 8.79307347, 8.78789465, 8.78268602,
       8.77744778, 8.7721801 , 8.7668832 , 8.76155726, 8.7562025 ,
       8.75081912, 8.74540733, 8.73996735, 8.73449939, 8.72900367,
       8.72348043, 8.71792987, 8.71235223, 8.70674774, 8.70111662,
       8.69545908, 8.68977538, 8.68406574, 8.67833041, 8.67256962,
       8.66678363, 8.66097269, 8.65513703, 8.64927693, 8.64339

In [ ]:
import matplotlib.pyplot as plt

fig1, axs1 = plt.subplots(1, 1, sharex=True, figsize=(5, 5))

axs1.plot( tdvp_ex1.operator_results()[0] , label="gamma = 0")
axs1.plot( magnetization_z , label="gamma = ex")

axs1.set_xlabel("Time $t$")
axs1.set_ylabel("Total Occupation ")
axs1.grid(True)
axs1.legend()

In [ ]:
times = tdvp_ex1.times()
tdvp_ex1.operator_results()[0]

In [ ]:
import matplotlib.pyplot as plt

def get_distinct_colors(n):
    """Generates a list of n distinct, visually appealing colors using colormap cyclically."""
    from matplotlib.cm import get_cmap
    cmap = get_cmap('tab20')  # Choose a colormap with more distinct colors
    return [cmap(i / n) for i in range(n)]

# Create a figure with a grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(20, 16))

# Set a title for the entire figure
plt.suptitle(
    r'$\hat{H}=\sum_{\langle i j\rangle}\left(J_x \hat{X}_i \hat{X}_j+J_y \hat{Y}_i \hat{Y}_j+J_z \hat{Z}_i \hat{Z}_j\right)+\sum_i h_z \hat{Z}_i$' + '\n' + 
    r'$(J_x,J_y,J_z,h_z,\gamma) = (0.9, 1, 1, 0, 1) $' + '\n' +
    r'$time\_step = 0.001$' + '\n' +
    r'$run\ time = 45 min $',
    fontsize=30, y=0.98
)

# Plot max bond dimension
axs[0, 0].plot(tdvp_ex1.max_bond_dim, linewidth=3)
axs[0, 0].set_xlabel("Time $t$", fontsize=15)
axs[0, 0].set_ylabel('Max Bond Dimension', fontsize=15)
axs[0, 0].tick_params(axis='both', which='major', labelsize=20)
axs[0, 0].grid(True)

# Plot total bond dimension
axs[0, 1].plot(tdvp_ex1.total_bond_dim, linewidth=3)
axs[0, 1].set_xlabel("Time $t$", fontsize=15)
axs[0, 1].set_ylabel('Total Bond Dimension', fontsize=15)
axs[0, 1].tick_params(axis='both', which='major', labelsize=20)
axs[0, 1].grid(True)

# Plot bond dimensions for each key
colors = get_distinct_colors(len(tdvp_ex1.bond_dims))
for i, (key, values) in enumerate(tdvp_ex1.bond_dims.items()):
    x = list(range(len(values)))
    axs[1, 0].plot(x, values, label=key, color=colors[i], linewidth=3)
axs[1, 0].legend(title='Bond Dimensions', loc='upper left', bbox_to_anchor=(1, 1), fontsize=14, title_fontsize=16)
axs[1, 0].set_xlabel("Time $t$", fontsize=18)
axs[1, 0].set_ylabel('Bond dimensions', fontsize=15)
axs[1, 0].set_title('Time (timestep = 0.01)', fontsize=15)
axs[1, 0].tick_params(axis='both', which='major', labelsize=15)
plt.subplots_adjust(right=0.8)

# Plot results_M
axs[1, 1].plot(times , tdvp_ex1.operator_results()[0], label="Total Magnetization", linewidth=3)
axs[1, 1].set_xlabel("Time $t$", fontsize=15)
axs[1, 1].set_ylabel("Total Magnetization", fontsize=15)
axs[1, 1].grid(True)
axs[1, 1].legend()

# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make room for the title

# Show the plot
plt.show()


In [ ]:
### (0, 0): (2, 2, 2) --> (2, 2, 2, 2)

shapes = {
    (0, 0): (2, 2, 2),
    (0, 1): (2, 2, 2),
    (0, 2): (2, 2, 2, 2),
    (0, 3): (2, 2),
    (1, 0): (2, 2),
    (1, 1): (2, 2, 2, 2),
    (1, 2): (2, 2, 2, 2),
    (1, 3): (2, 2),
    (2, 0): (2, 2),
    (2, 1): (2, 2, 2, 2),
    (2, 2): (2, 2, 2, 2),
    (2, 3): (2, 2),
    (3, 0): (2, 2),
    (3, 1): (2, 2, 2),
    (3, 2): (2, 2, 2, 2),
    (3, 3): (2, 2)
}

sites = {
    (i, j): ptn.random_tensor_node(shapes[(i, j)], identifier=f"Site({i},{j})") for i in range(4) for j in range(4)
}

ttn = ptn.TreeTensorNetworkState()

ttn.add_root(sites[(0, 0)][0], sites[(0, 0)][1])

connections = [
    ((0, 0), (0, 1), 0, 0),
    ((0, 1), (0, 2), 1, 0),
    ((0, 2), (0, 3), 1, 0),
    ((0, 2), (1, 2), 2, 0),
    ((1, 2), (1, 3), 1, 0),
    ((1, 2), (1, 1), 2, 0),
    ((1, 1), (1, 0), 1, 0),
    ((1, 1), (2, 1), 2, 0),
    ((2, 1), (2, 0), 1, 0),
    ((2, 1), (2, 2), 2, 0),
    ((2, 2), (2, 3), 1, 0),
    ((2, 2), (3, 2), 2, 0),
    ((3, 2), (3, 3), 1, 0),
    ((3, 2), (3, 1), 2, 0),
    ((3, 1), (3, 0), 1, 0),
    ]

for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Site({parent[0]},{parent[1]})"
    child_id = f"Site({child[0]},{child[1]})"
    ttn.add_child_to_parent(sites[child][0], sites[child][1], child_leg, parent_id, parent_leg)

ttn = product_state(ttn , bond_dim= 2, physical_dim = 2)  

nodes = {
    (i, j): (ptn.Node(tensor=ttn.tensors[f"Site({i},{j})"].conj() , identifier=f"Node({i},{j})"), ttn.tensors[f"Site({i},{j})"].conj()) for i in range(4) for j in range(4)
}

ttn.add_child_to_parent(nodes[(0,0)][0], nodes[(0,0)][1], 1, "Site(0,0)", 1)

### (0,0) : 0,1 --> 1,2

connections = [
    ((0, 0), (0, 1), 1, 0),
    ((0, 1), (0, 2), 1, 0),
    ((0, 2), (0, 3), 1, 0),
    ((0, 2), (1, 2), 2, 0),
    ((1, 2), (1, 3), 1, 0),
    ((1, 2), (1, 1), 2, 0),
    ((1, 1), (1, 0), 1, 0),
    ((1, 1), (2, 1), 2, 0),
    ((2, 1), (2, 0), 1, 0),
    ((2, 1), (2, 2), 2, 0),
    ((2, 2), (2, 3), 1, 0),
    ((2, 2), (3, 2), 2, 0),
    ((3, 2), (3, 3), 1, 0),
    ((3, 2), (3, 1), 2, 0),
    ((3, 1), (3, 0), 1, 0),
    ]

for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Node({parent[0]},{parent[1]})"
    child_id = f"Node({child[0]},{child[1]})"
    ttn.add_child_to_parent(nodes[child][0], nodes[child][1], child_leg, parent_id, parent_leg)

In [ ]:
# Config : Lindblad = True
# time_evolve with exponent = hamiltonian * time_difference
# evaluate_operator with operator_expectation_value_Lindblad(operator)
# normalize_ttn_Lindblad after each run_one_time_step_ex

tdvp_ex1 = ptn.SecondOrderOneSiteTDVP(initial_state = ttn,
                                     hamiltonian = L_fancy,
                                     time_step_size = 0.01,
                                     final_time = 4,
                                     operators = M,
                                     num_vecs = 3,
                                     tau = 1e-3,
                                     SVDParameters = ptn.SVDParameters(max_bond_dim = np.inf , rel_tol= -np.inf , total_tol = -np.inf),
                                     expansion_steps = 15,
                                     t3n_dict = {'Site(0,2)': 'Site(0,1)',
                                                'Site(1,2)': 'Site(1,1)',
                                                'Site(1,1)': 'Site(2,1)',
                                                'Site(2,1)': 'Site(2,2)',
                                                'Site(2,2)': 'Site(2,1)',
                                                'Site(3,2)': 'Site(3,3)',

                                                'Node(0,2)': 'Node(0,1)',
                                                'Node(1,2)': 'Node(1,1)',
                                                'Node(1,1)': 'Node(2,1)',
                                                'Node(2,1)': 'Node(2,2)',
                                                'Node(2,2)': 'Node(2,1)',
                                                'Node(3,2)': 'Node(3,3)'},

                                     Lanczos_threshold = np.inf,
                                     k_fraction = 0.2, 
                                     validity_fraction = 1, 
                                     increase_fraction = 0.3,
                                     max_iter = 1, 

                                     initial_tol= 1e-16,
                                     tol_step= 10,
                                     rel_tot_bond = 30,
                                     max_bond= 220,
                                     norm_tol = 0,
                                     KrylovBasisMode = ptn.KrylovBasisMode.apply_1st_order_expansion,
                                     config = ptn.TTNTimeEvolutionConfig(record_bond_dim=True,
                                                                         Lindblad = True,
                                                                         T3NS= False) )

In [ ]:
from __future__ import annotations
from typing import List, Union
import copy
from tqdm import tqdm
import numpy as np
from dataclasses import replace
from ...operators.tensorproduct import TensorProduct
from ...ttno.ttno_class import TTNO
from ...ttns import (TreeTensorNetworkState , 
                     normalize_ttn_Lindblad_1 , 
                        normalize_ttn_Lindblad_1_conj ,
                     normalize_ttn_Lindblad_3 , 
                        normalize_ttn_Lindblad_3_conj ,
                     normalize_ttn_Lindblad_4 , 
                     normalize_ttn_Lindblad_5,
                        normalize_ttn_Lindblad_5_conj,
                     normalize_ttn_Lindblad_XX ,  )
from ..ttn_time_evolution import TTNTimeEvolutionConfig
from ..Subspace_expansion import expand_subspace , KrylovBasisMode , max_two_neighbour_form , original_form
from ...util.tensor_splitting import SplitMode , SVDParameters
from ...core.canonical_form import adjust_ttn1_structure_to_ttn2 , adjust_ttno_structure_to_ttn
from pytreenet.contractions.state_operator_contraction import adjust_operator_to_ket , adjust_bra_to_ket
from pytreenet.contractions.state_state_contraction import contract_ttn_Lindblad
from .onesitetdvp import OneSiteTDVP
from ...contractions.tree_cach_dict import PartialTreeCachDict
from ...time_evolution.time_evo_util.update_path import TDVPUpdatePathFinder
class SecondOrderOneSiteTDVP(OneSiteTDVP):
    """
    The first order one site TDVP algorithm.

    This means we have second order Trotter splitting for the time evolution:
      exp(At+Bt) approx exp(At/2)*exp(Bt/2)*exp(Bt/2)*exp(At/2)

    Has the same attributes as the TDVP-Algorithm clas with two additions.

    Attributes:
        backwards_update_path (List[str]): The update path that traverses
            backwards.
        backwards_orth_path (List[List[str]]): The orthogonalisation paths for
            the backwards run.
    """

    def __init__(self, 
                 initial_state: TreeTensorNetworkState,
                 hamiltonian: TTNO, 
                 time_step_size: float, 
                 final_time: float,
                 operators: Union[TensorProduct, List[TensorProduct]],

                 num_vecs: int , 
                 tau: float, 
                 SVDParameters : SVDParameters,
                 expansion_steps: int = 10,
                 t3n_dict: dict = None,

                 Lanczos_threshold : float = 10,
                 k_fraction : float = 0.6, 
                 validity_fraction : float = 0.8, 
                 increase_fraction : float = 0.3, 
                 max_iter : int = 10,  

                 initial_tol: float = 1e-20,
                 tol_step: float = 10, 
                 rel_tot_bond : int = 30,
                 max_bond: int = 100,
                 norm_tol: int = 0,
                 KrylovBasisMode : KrylovBasisMode = KrylovBasisMode.apply_ham,                  
                 config: Union[TTNTimeEvolutionConfig,None] = None) -> None:
        """
        Initialize the second order one site TDVP algorithm.

        Args:
            initial_state (TreeTensorNetworkState): The initial state of the
                system.
            hamiltonian (TTNO): The Hamiltonian of the system.
            time_step_size (float): The time step size.
            final_time (float): The final time of the evolution.
            operators (Union[TensorProduct, List[TensorProduct]]): The operators
                for which the expectation values are calculated.
            config (Union[TTNTimeEvolutionConfig,None], optional): The time
                evolution configuration. Defaults to None.
        """
        super().__init__(initial_state, hamiltonian,
                         time_step_size, final_time, operators, num_vecs, 
                         tau,
                         SVDParameters,
                         expansion_steps,
                         initial_tol,
                         tol_step,
                         rel_tot_bond,
                         max_bond,
                         KrylovBasisMode,  
                         config)
        self.t3n_dict = t3n_dict
        self.Lanczos_threshold = Lanczos_threshold
        self.k_fraction = k_fraction
        self.validity_fraction = validity_fraction
        self.increase_fraction = increase_fraction
        self.max_iter = max_iter
        self.norm_tol = norm_tol

        if self.T3NS :
           self._init_two_neighbour_form() 
        else:
            self.hamiltonian = adjust_ttno_structure_to_ttn(self.hamiltonian , self.state)
            self.update_path = TDVPUpdatePathFinder(self.state).find_path()
            self.orthogonalization_path = self._find_tdvp_orthogonalization_path(self.update_path) 
            self.backwards_update_path = self._init_second_order_update_path()
            self.backwards_orth_path = self._init_second_order_orth_path() 
            self._orthogonalize_init()
            self.partial_tree_cache = PartialTreeCachDict()
            self._init_partial_tree_cache()           

    def _init_two_neighbour_form(self):
        """
        Transform the state, Hamiltonian and operators into the max two neighbour form.
        """
        self.hamiltonian = adjust_ttno_structure_to_ttn(self.hamiltonian , self.state)
        self.state , dict1 = max_two_neighbour_form(self.state)
        self.hamiltonian , _ = max_two_neighbour_form(self.hamiltonian , dict1)
        list = []
        for operators in self.operators: 
            operator , _ = max_two_neighbour_form(operators,dict1)
            list.append(operator)
        self.operators = list

        self.hamiltonian = adjust_ttno_structure_to_ttn(self.hamiltonian , self.state)
        self.update_path = TDVPUpdatePathFinder(self.state).find_path()
        self.orthogonalization_path = self._find_tdvp_orthogonalization_path(self.update_path) 
        self.backwards_update_path = self._init_second_order_update_path()
        self.backwards_orth_path = self._init_second_order_orth_path() 
        self.two_neighbour_form_dict = dict1
        self._orthogonalize_init()
        self.partial_tree_cache = PartialTreeCachDict()
        self._init_partial_tree_cache()
        
    
    def _init_second_order_update_path(self) -> List[str]:
        """
        Find the update path that traverses backwards.
        """
        return list(reversed(self.update_path))

    def _init_second_order_orth_path(self) -> List[List[str]]:
        """
        Find the orthogonalisation paths for the backwards run.
        """
        back_orthogonalization_path = []
        for i, node_id in enumerate(self.backwards_update_path[1:-1]):
            current_path = self.state.path_from_to(node_id,
                                                   self.backwards_update_path[i+2])
            current_path = current_path[:-1]
            back_orthogonalization_path.append(current_path)
        back_orthogonalization_path.append([self.backwards_update_path[-1]])
        return back_orthogonalization_path

    def _update_forward_site_and_link(self, node_id: str,
                                      next_node_id: str):
        """
        Run the forward update with half time step.

        First the site tensor is updated and then the link tensor.

        Args:
            node_id (str): The identifier of the site to be updated.
            next_node_id (str): The other node of the link to be updated.
        """
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id,
                          time_step_factor=0.5)
        self._update_link(node_id, next_node_id,
                          time_step_factor=0.5)

    def forward_sweep(self):
        """
        Perform the forward sweep through the state.
        """
        for i, node_id in enumerate(self.update_path[:-1]):
            # Orthogonalize
            if i>0:
                self._move_orth_and_update_cache_for_path(self.orthogonalization_path[i-1])
            # Select Next Node
            next_node_id = self.orthogonalization_path[i][0]
            # Update
            self._update_forward_site_and_link(node_id, next_node_id)

    def _final_forward_update(self):
        """
        Perform the final forward update. 
        
        To save some computation, the update is performed with a full time
        step. Since the first update backwards occurs on the same node. We
        also do not need to update any link tensors.
        """
        node_id = self.update_path[-1]
        assert node_id == self.backwards_update_path[0]
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id)

    def _update_first_backward_link(self):
        """
        Update the link between the first and second node in the backwards
        update path with a half time step.
        
        We have already updated the first site on the backwards update path
        and the link will always be next to it, so the orthogonality center
        is already at the correct position.
        """
        next_node_id = self.backwards_update_path[1]
        self._update_link(self.state.orthogonality_center_id,
                          next_node_id,
                          time_step_factor=0.5)

    def _normal_backward_update(self, node_id: str,
                                update_index: int):
        """
        The normal way to make a backwards update.
        
        First the site tensor is updated. Then the orthogonality center is
        moved, if needed. Finally the link tensor between the new
        orthogonality center and the next node is updated. 
        
        Args:
            node_id (str): The identifier of the site to be updated.
            update_index (int): The index of the update in the backwards
                update path.
        """
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id, time_step_factor=0.5)
        new_orth_center = self.backwards_orth_path[update_index-1]
        self._move_orth_and_update_cache_for_path(new_orth_center)
        next_node_id = self.backwards_update_path[update_index+1]
        self._update_link(self.state.orthogonality_center_id,
                          next_node_id,
                          time_step_factor=0.5)

    def _final_backward_update(self):
        """
        Perform the final backward update.
        
        Since this is the last node that needs updating, no link update is
        required afterwards.
        """
        node_id = self.backwards_update_path[-1]
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id, time_step_factor=0.5)

    def backward_sweep(self):
        """
        Perform the backward sweep through the state.
        """
        self._update_first_backward_link()
        for i, node_id in enumerate(self.backwards_update_path[1:-1]):
            self._normal_backward_update(node_id, i+1)
        self._final_backward_update()

    def run_one_time_step(self):
        """
        Run a single second order time step.
        
        This mean we run a full forward and a full backward sweep through the
        tree.
        """
        self.forward_sweep()
        self._final_forward_update()
        self.backward_sweep()


    def forward_sweep_ex(self):
        """
        Perform the forward sweep through the state.
        """
        for i, node_id in enumerate(self.update_path[:-1]):
            # Orthogonalize
            if i>0:
                self._move_orth_and_update_cache_for_path(self.orthogonalization_path[i-1])
            # Select Next Node
            next_node_id = self.orthogonalization_path[i][0]
            # Update
            self._update_forward_site_and_link(node_id, next_node_id)

    def run_one_time_step_ex(self):
        """
        Run a single second order time step.
        
        This mean we run a full forward and a full backward sweep through the
        tree.
        """

        self.forward_sweep_ex()
        self._final_forward_update()
        self.backward_sweep()  
    

    # EXPANDS with a predefined T3NS
    def run_ex(self, evaluation_time: Union[int,"inf"] = 1, filepath: str = "",
            pgbar: bool = True,):
        """
        Runs this time evolution algorithm for the given parameters.

        The desired operator expectation values are evaluated and saved.

        Args:
            evaluation_time (int, optional): The difference in time steps after which
                to evaluate the operator expectation values, e.g. for a value of 10
                the operators are evaluated at time steps 0,10,20,... If it is set to
                "inf", the operators are only evaluated at the end of the time.
                Defaults to 1.
            filepath (str, optional): If results are to be saved in an external file,
                the path to that file can be specified here. Defaults to "".
            pgbar (bool, optional): Toggles the progress bar. Defaults to True.
        """
        should_expand = True
        self._init_results(evaluation_time)
        assert self._results is not None
        tol = self.initial_tol
        I = TTNO.Identity(self.state)
        t3no , _ = max_two_neighbour_form(self.hamiltonian , self.t3n_dict)

        for i in tqdm(range(self.num_time_steps + 1), disable=not pgbar):
            ttn_copy_1 = deepcopy(self.state)

            I_ex = ttn_copy_1.operator_expectation_value_Lindblad(I)              
            if np.abs(np.abs(I_ex) - 1)  > self.norm_tol:
                orth_center_id_1 = self.state.root_id
                orth_center_id_2 = orth_center_id_1.replace('Site', 'Node')
                #self.state = normalize_ttn_Lindblad_3(ttn_copy_1 , orth_center_id_1 , orth_center_id_2) # better than 1 and 4
                #self.state = normalize_ttn_Lindblad_1_conj(ttn_copy_1) # better than 1 and 4

                update_path_0 = self.update_path[0]
                self.state = normalize_ttn_Lindblad_4(ttn_copy_1 , update_path_0)                 
                norm = self.state.operator_expectation_value_Lindblad(I)
                #print("Norm :" ,norm, np.abs(norm))
            else:
                self.state = ttn_copy_1  
                norm1 = ttn_copy_1.operator_expectation_value_Lindblad(I)
                norm2 = contract_ttn_Lindblad(ttn_copy_1)
                print("Norm1 :" ,norm1, np.abs(norm1))
                print("Norm2 :" ,norm2, np.abs(norm2))

            if evaluation_time != "inf" and i % evaluation_time == 0 and len(self._results) > 0:
                index = i // evaluation_time
                current_results = self.evaluate_operators() / norm
                print("M :" , current_results[0])
                self._results[0:-1, index] = current_results
                # Save current time
                self._results[-1, index] = i*self.time_step_size  

            self._orthogonalize_init(force_new=True)
            self.partial_tree_cache = PartialTreeCachDict()
            self._init_partial_tree_cache() 
            self.run_one_time_step_ex() 

            ########### EXAPNSION ###########
            before_ex_total_bond_ttn = self.state.total_bond_dim()
            
            if (i+1) % (self.expansion_steps+1) == 0 and should_expand:  
                ######## T3NS ########
                ttn = deepcopy(self.state)
                t3n , _  = max_two_neighbour_form(ttn , self.t3n_dict)
                
                adjust_operator_to_ket(t3no,t3n)
                adjust_bra_to_ket(t3n)
                before_ex_total_bond_t3ns = t3n.total_bond_dim()

                self.SVDParameters = replace(self.SVDParameters, max_bond_dim = t3n.max_bond_dim())
                print("SVD MAX :" , self.SVDParameters.max_bond_dim)

                print("tol :" , tol)       
                state_ex_t3n = expand_subspace(t3n, 
                                            t3no, 
                                            self.num_vecs, 
                                            self.tau, 
                                            self.SVDParameters, 
                                            tol, 
                                            self.Lanczos_threshold, 
                                            self.k_fraction, 
                                            self.validity_fraction, 
                                            self.increase_fraction,
                                            self.max_iter,
                                            self.KrylovBasisMode)
                after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                state_ex_ttn = original_form(state_ex_t3n , self.t3n_dict)
                expanded_dim_tot = state_ex_ttn.total_bond_dim() - ttn.total_bond_dim()
                if  expanded_dim_tot > self.rel_tot_bond:
                    print("expanded_dim_tot :" , expanded_dim_tot)
                    A = True
                    # tol_prime = tol
                    for _ in range(10):
                        if A:
                            tol *= self.tol_step
                            print("1) tol" , tol)                            
                            state_ex_t3n_prime = expand_subspace(t3n, 
                                                                t3no, 
                                                                self.num_vecs, 
                                                                self.tau, 
                                                                self.SVDParameters, 
                                                                tol, 
                                                                self.Lanczos_threshold, 
                                                                self.k_fraction, 
                                                                self.validity_fraction, 
                                                                self.increase_fraction,
                                                                self.max_iter,
                                                                self.KrylovBasisMode)
                            after_ex_total_bond_t3ns = state_ex_t3n_prime.total_bond_dim()
                            state_ex_ttn = original_form(state_ex_t3n_prime , self.t3n_dict)
                            expanded_dim_tot = state_ex_ttn.total_bond_dim() - ttn.total_bond_dim()
                            print("2) expanded_dim :" , expanded_dim_tot)
                            if expanded_dim_tot < 0:
                                state_ex_ttn = ttn
                                tol /= self.tol_step
                                A = False
                            elif expanded_dim_tot < self.rel_tot_bond :  
                                A = False  
                self._orthogonalize_init(force_new=True)                     

                if self.max_bond < state_ex_ttn.total_bond_dim():
                    print(self.max_bond , state_ex_ttn.total_bond_dim()) 
                    state_ex_ttn = ttn
                    should_expand = False
                    print("3")

                if state_ex_ttn.total_bond_dim() - before_ex_total_bond_ttn <= 0:
                   state_ex_ttn = ttn
                   tol /= self.tol_step
                   print(state_ex_ttn.total_bond_dim() , before_ex_total_bond_ttn)
                   print("4")      

                self.state = state_ex_ttn
                after_ex_total_bond_ttn = self.state.total_bond_dim()

                expanded_dim_total_bond_ttn = after_ex_total_bond_ttn - before_ex_total_bond_ttn
                expanded_dim_total_bond_t3ns = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns

                if self.max_bond < after_ex_total_bond_ttn:
                    print("END :" , after_ex_total_bond_ttn)
                    should_expand = False      
                
                print("expanded_dim T3NS:" , expanded_dim_total_bond_t3ns)      
                print("T3NS:" , before_ex_total_bond_t3ns , "--->" , after_ex_total_bond_t3ns)   

                print("expanded_dim TTN:" , expanded_dim_total_bond_ttn)
                print("TTN:" , before_ex_total_bond_ttn , "--->" , after_ex_total_bond_ttn)    
 
            ##################################
                 
            self.record_bond_dimensions()
                    
        if evaluation_time == "inf":
            current_results = self.evaluate_operators()
            self._results[0:-1, 0] = current_results
            self._results[-1, 0] = i*self.time_step_size
        if filepath != "":
            self.save_results_to_file(filepath)         



    # RUN entirly in T3NS form                
    def run_ex_t3n(self, evaluation_time: Union[int,"inf"] = 1, filepath: str = "",
            pgbar: bool = True,):
        """
        Runs this time evolution algorithm for the given parameters.

        The desired operator expectation values are evaluated and saved.

        Args:
            evaluation_time (int, optional): The difference in time steps after which
                to evaluate the operator expectation values, e.g. for a value of 10
                the operators are evaluated at time steps 0,10,20,... If it is set to
                "inf", the operators are only evaluated at the end of the time.
                Defaults to 1.
            filepath (str, optional): If results are to be saved in an external file,
                the path to that file can be specified here. Defaults to "".
            pgbar (bool, optional): Toggles the progress bar. Defaults to True.
        """
        self._init_two_neighbour_form() 
        should_expand = True
        self._init_results(evaluation_time)
        assert self._results is not None
        tol = self.initial_tol
        I = TTNO.Identity(self.state)

        for i in tqdm(range(self.num_time_steps + 1), disable=not pgbar):
            t3n_copy_1 = deepcopy(self.state)

            if  self.normalize:
                #orth_center_id_1 = self.state.root_id
                #orth_center_id_2 = orth_center_id_1.replace('Site', 'Node')
                #self.state = normalize_ttn_Lindblad_3_conj(t3n_copy_1 , orth_center_id_1 , orth_center_id_2) # better than 1 and 4
                #self._orthogonalize_init(force_new=True)

                update_path_0 = self.update_path[0]
                self.state = normalize_ttn_Lindblad_4(t3n_copy_1 , update_path_0) 
                
                self.partial_tree_cache = PartialTreeCachDict()
                self._init_partial_tree_cache()                 
                norm = self.state.operator_expectation_value_Lindblad(I)
                #print("Norm :" ,norm, np.abs(norm))
            else:
                self.state = t3n_copy_1  
                norm = t3n_copy_1.operator_expectation_value_Lindblad(I)
                #print("Norm :" ,norm, np.abs(norm))

            if evaluation_time != "inf" and i % evaluation_time == 0 and len(self._results) > 0:
                index = i // evaluation_time
                current_results = self.evaluate_operators()
                #print("H :" , self.evaluate_operators()[0] / norm)
                self._results[0:-1, index] = current_results
                # Save current time
                self._results[-1, index] = i*self.time_step_size  

            self.run_one_time_step_ex() 

            ########### EXAPNSION ###########
            before_ex_total_bond_t3ns = self.state.total_bond_dim()
            
            if (i+1) % (self.expansion_steps+1) == 0 and should_expand:  
                
                ######## T3NS ########
                t3n = deepcopy(self.state)
                t3no = deepcopy(self.hamiltonian)
                        
                adjust_operator_to_ket(t3no,t3n)
                adjust_bra_to_ket(t3n)
                print("tol :" , tol)   
                self.SVDParameters = replace(self.SVDParameters, max_bond_dim = t3n.max_bond_dim())
                print("max_bond_dim :" , self.SVDParameters.max_bond_dim)
                state_ex_t3n = expand_subspace(t3n, 
                                            t3no, 
                                            self.num_vecs, 
                                            self.tau, 
                                            self.SVDParameters, 
                                            tol, 
                                            self.Lanczos_threshold, 
                                            self.k_fraction, 
                                            self.validity_fraction, 
                                            self.increase_fraction,
                                            self.max_iter,
                                            self.KrylovBasisMode)
                after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                expanded_dim_tot = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns
                if expanded_dim_tot <= 0:
                    state_ex_t3n = t3n
                    tol /= self.tol_step

                if  expanded_dim_tot > self.rel_tot_bond:
                    print("expanded_dim_tot :" , expanded_dim_tot)
                    A = True
                    # tol_prime = tol
                    for _ in range(10):
                        if A:
                            tol *= self.tol_step
                            print("1) tol" , tol)                            
                            state_ex_t3n = expand_subspace(t3n, 
                                                                t3no, 
                                                                self.num_vecs, 
                                                                self.tau, 
                                                                self.SVDParameters, 
                                                                tol, 
                                                                self.Lanczos_threshold, 
                                                                self.k_fraction, 
                                                                self.validity_fraction, 
                                                                self.increase_fraction,
                                                                self.max_iter,
                                                                self.KrylovBasisMode)
                            after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                            expanded_dim_tot = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns

                            print("2) expanded_dim :" , expanded_dim_tot)
                            if expanded_dim_tot <= 0:
                                state_ex_t3n = t3n
                                tol /= self.tol_step
                                A = False
                            elif expanded_dim_tot < self.rel_tot_bond :  
                                A = False                

                if self.max_bond < state_ex_t3n.total_bond_dim():
                    print(self.max_bond , state_ex_t3n.total_bond_dim()) 
                    state_ex_t3n = t3n
                    should_expand = False
                    print("3") 

                self.state = state_ex_t3n
                self._orthogonalize_init(force_new=True)

                after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                expanded_dim_tot = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns 
                                   
                print("expanded_dim T3NS:" , expanded_dim_tot)      
                print("T3NS:" , before_ex_total_bond_t3ns , "--->" , after_ex_total_bond_t3ns)      
 
            ##################################
                 
            self.record_bond_dimensions()
                    
        if evaluation_time == "inf":
            current_results = self.evaluate_operators()
            self._results[0:-1, 0] = current_results
            self._results[-1, 0] = i*self.time_step_size
        if filepath != "":
            self.save_results_to_file(filepath)   


In [ ]:
from __future__ import annotations
from typing import List, Union
import copy
from tqdm import tqdm
import numpy as np
from dataclasses import replace
from ...operators.tensorproduct import TensorProduct
from ...ttno.ttno_class import TTNO
from ...ttns import (TreeTensorNetworkState , 
                     normalize_ttn_Lindblad_1 , 
                        normalize_ttn_Lindblad_1_conj ,
                     normalize_ttn_Lindblad_3 , 
                        normalize_ttn_Lindblad_3_conj ,
                     normalize_ttn_Lindblad_4 , 
                     normalize_ttn_Lindblad_5,
                        normalize_ttn_Lindblad_5_conj,
                     normalize_ttn_Lindblad_XX ,  )
from ..ttn_time_evolution import TTNTimeEvolutionConfig
from ..Subspace_expansion import expand_subspace , KrylovBasisMode , max_two_neighbour_form , original_form
from ...util.tensor_splitting import SplitMode , SVDParameters
from ...core.canonical_form import adjust_ttn1_structure_to_ttn2 , adjust_ttno_structure_to_ttn
from pytreenet.contractions.state_operator_contraction import adjust_operator_to_ket , adjust_bra_to_ket
from pytreenet.contractions.state_state_contraction import contract_ttn_Lindblad
from .onesitetdvp import OneSiteTDVP
from ...contractions.tree_cach_dict import PartialTreeCachDict
from ...time_evolution.time_evo_util.update_path import TDVPUpdatePathFinder

                       


class SecondOrderOneSiteTDVP(OneSiteTDVP):
    """
    The first order one site TDVP algorithm.

    This means we have second order Trotter splitting for the time evolution:
      exp(At+Bt) approx exp(At/2)*exp(Bt/2)*exp(Bt/2)*exp(At/2)

    Has the same attributes as the TDVP-Algorithm clas with two additions.

    Attributes:
        backwards_update_path (List[str]): The update path that traverses
            backwards.
        backwards_orth_path (List[List[str]]): The orthogonalisation paths for
            the backwards run.
    """

    def __init__(self, 
                 initial_state: TreeTensorNetworkState,
                 hamiltonian: TTNO, 
                 time_step_size: float, 
                 final_time: float,
                 operators: Union[TensorProduct, List[TensorProduct]],

                 num_vecs: int , 
                 tau: float, 
                 SVDParameters : SVDParameters,
                 expansion_steps: int = 10,
                 t3n_dict: dict = None,

                 Lanczos_threshold : float = 10,
                 k_fraction : float = 0.6, 
                 validity_fraction : float = 0.8, 
                 increase_fraction : float = 0.3, 
                 max_iter : int = 10,  

                 initial_tol: float = 1e-20,
                 tol_step: float = 10, 
                 rel_tot_bond : int = 30,
                 max_bond: int = 100,
                 KrylovBasisMode : KrylovBasisMode = KrylovBasisMode.apply_ham,                  
                 config: Union[TTNTimeEvolutionConfig,None] = None) -> None:
        """
        Initialize the second order one site TDVP algorithm.

        Args:
            initial_state (TreeTensorNetworkState): The initial state of the
                system.
            hamiltonian (TTNO): The Hamiltonian of the system.
            time_step_size (float): The time step size.
            final_time (float): The final time of the evolution.
            operators (Union[TensorProduct, List[TensorProduct]]): The operators
                for which the expectation values are calculated.
            config (Union[TTNTimeEvolutionConfig,None], optional): The time
                evolution configuration. Defaults to None.
        """
        super().__init__(initial_state, hamiltonian,
                         time_step_size, final_time, operators, num_vecs, 
                         tau,
                         SVDParameters,
                         expansion_steps,
                         initial_tol,
                         tol_step,
                         rel_tot_bond,
                         max_bond,
                         KrylovBasisMode,  
                         config)
        self.t3n_dict = t3n_dict
        self.Lanczos_threshold = Lanczos_threshold
        self.k_fraction = k_fraction
        self.validity_fraction = validity_fraction
        self.increase_fraction = increase_fraction
        self.max_iter = max_iter

        if self.T3NS :
           self._init_two_neighbour_form() 
        else:
            self.adjust_to_initial_structure()
            self.update_path = TDVPUpdatePathFinder(self.state).find_path()
            self.orthogonalization_path = self._find_tdvp_orthogonalization_path(self.update_path) 
            self.backwards_update_path = self._init_second_order_update_path()
            self.backwards_orth_path = self._init_second_order_orth_path() 
            self._orthogonalize_init()
            self.partial_tree_cache = PartialTreeCachDict()
            self._init_partial_tree_cache()           

    def _init_two_neighbour_form(self):
        """
        Transform the state, Hamiltonian and operators into the max two neighbour form.
        """
        self.adjust_to_initial_structure()
        self.state , dict1 = max_two_neighbour_form(self.state)
        self.hamiltonian , _ = max_two_neighbour_form(self.hamiltonian , dict1)
        list = []
        for operators in self.operators: 
            operator , _ = max_two_neighbour_form(operators,dict1)
            list.append(operator)
        self.operators = list

        self.adjust_to_initial_structure()
        self.update_path = TDVPUpdatePathFinder(self.state).find_path()
        self.orthogonalization_path = self._find_tdvp_orthogonalization_path(self.update_path) 
        self.backwards_update_path = self._init_second_order_update_path()
        self.backwards_orth_path = self._init_second_order_orth_path() 
        self.two_neighbour_form_dict = dict1
        self._orthogonalize_init()
        self.partial_tree_cache = PartialTreeCachDict()
        self._init_partial_tree_cache()
        
    
    def _init_second_order_update_path(self) -> List[str]:
        """
        Find the update path that traverses backwards.
        """
        return list(reversed(self.update_path))

    def _init_second_order_orth_path(self) -> List[List[str]]:
        """
        Find the orthogonalisation paths for the backwards run.
        """
        back_orthogonalization_path = []
        for i, node_id in enumerate(self.backwards_update_path[1:-1]):
            current_path = self.state.path_from_to(node_id,
                                                   self.backwards_update_path[i+2])
            current_path = current_path[:-1]
            back_orthogonalization_path.append(current_path)
        back_orthogonalization_path.append([self.backwards_update_path[-1]])
        return back_orthogonalization_path

    def _update_forward_site_and_link(self, node_id: str,
                                      next_node_id: str):
        """
        Run the forward update with half time step.

        First the site tensor is updated and then the link tensor.

        Args:
            node_id (str): The identifier of the site to be updated.
            next_node_id (str): The other node of the link to be updated.
        """
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id,
                          time_step_factor=0.5)
        self._update_link(node_id, next_node_id,
                          time_step_factor=0.5)

    def forward_sweep(self):
        """
        Perform the forward sweep through the state.
        """
        for i, node_id in enumerate(self.update_path[:-1]):
            # Orthogonalize
            if i>0:
                self._move_orth_and_update_cache_for_path(self.orthogonalization_path[i-1])
            # Select Next Node
            next_node_id = self.orthogonalization_path[i][0]
            # Update
            self._update_forward_site_and_link(node_id, next_node_id)

    def _final_forward_update(self):
        """
        Perform the final forward update. 
        
        To save some computation, the update is performed with a full time
        step. Since the first update backwards occurs on the same node. We
        also do not need to update any link tensors.
        """
        node_id = self.update_path[-1]
        assert node_id == self.backwards_update_path[0]
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id)

    def _update_first_backward_link(self):
        """
        Update the link between the first and second node in the backwards
        update path with a half time step.
        
        We have already updated the first site on the backwards update path
        and the link will always be next to it, so the orthogonality center
        is already at the correct position.
        """
        next_node_id = self.backwards_update_path[1]
        self._update_link(self.state.orthogonality_center_id,
                          next_node_id,
                          time_step_factor=0.5)

    def _normal_backward_update(self, node_id: str,
                                update_index: int):
        """
        The normal way to make a backwards update.
        
        First the site tensor is updated. Then the orthogonality center is
        moved, if needed. Finally the link tensor between the new
        orthogonality center and the next node is updated. 
        
        Args:
            node_id (str): The identifier of the site to be updated.
            update_index (int): The index of the update in the backwards
                update path.
        """
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id, time_step_factor=0.5)
        new_orth_center = self.backwards_orth_path[update_index-1]
        self._move_orth_and_update_cache_for_path(new_orth_center)
        next_node_id = self.backwards_update_path[update_index+1]
        self._update_link(self.state.orthogonality_center_id,
                          next_node_id,
                          time_step_factor=0.5)

    def _final_backward_update(self):
        """
        Perform the final backward update.
        
        Since this is the last node that needs updating, no link update is
        required afterwards.
        """
        node_id = self.backwards_update_path[-1]
        assert self.state.orthogonality_center_id == node_id
        self._update_site(node_id, time_step_factor=0.5)

    def backward_sweep(self):
        """
        Perform the backward sweep through the state.
        """
        self._update_first_backward_link()
        for i, node_id in enumerate(self.backwards_update_path[1:-1]):
            self._normal_backward_update(node_id, i+1)
        self._final_backward_update()

    def forward_sweep_ex(self):
        """
        Perform the forward sweep through the state.
        """
        for i, node_id in enumerate(self.update_path[:-1]):
            # Orthogonalize
            if i>0:
                self._move_orth_and_update_cache_for_path(self.orthogonalization_path[i-1])
            # Select Next Node
            next_node_id = self.orthogonalization_path[i][0]
            # Update
            self._update_forward_site_and_link(node_id, next_node_id)

    def create_temp_copy(self):
        """
        Creates a temporary copy of the object with deep copies of state and cache.
        """
        temp_self = copy.copy(self)
        temp_self.state = copy.deepcopy(self.state)
        return temp_self

    def run_one_time_step(self):
        # remove self._orthogonalize_init() and self.partial_tree_cache in TDVPAlgorithm
        temp_self = self.create_temp_copy()
        temp_self._orthogonalize_init()
        temp_self.adjust_to_initial_structure()
        temp_self._init_partial_tree_cache()

        temp_self.forward_sweep()
        temp_self._final_forward_update()
        temp_self.backward_sweep()
        temp_self.adjust_to_initial_structure()

        orth_center_id_1 = temp_self.state.root_id
        orth_center_id_2 = orth_center_id_1.replace('Site', 'Node')
        temp_self.state = normalize_ttn_Lindblad_3_conj(self.state , orth_center_id_1 , orth_center_id_2 )
        self.state = temp_self.state
        self.adjust_to_initial_structure() 
    

    # EXPANDS with a predefined T3NS
    def run_ex(self, evaluation_time: Union[int,"inf"] = 1, filepath: str = "",
            pgbar: bool = True,):
        """
        Runs this time evolution algorithm for the given parameters.

        The desired operator expectation values are evaluated and saved.

        Args:
            evaluation_time (int, optional): The difference in time steps after which
                to evaluate the operator expectation values, e.g. for a value of 10
                the operators are evaluated at time steps 0,10,20,... If it is set to
                "inf", the operators are only evaluated at the end of the time.
                Defaults to 1.
            filepath (str, optional): If results are to be saved in an external file,
                the path to that file can be specified here. Defaults to "".
            pgbar (bool, optional): Toggles the progress bar. Defaults to True.
        """
        should_expand = True
        self._init_results(evaluation_time)
        assert self._results is not None
        tol = self.initial_tol
        I = TTNO.Identity(self.state)
        t3no , _ = max_two_neighbour_form(self.hamiltonian , self.t3n_dict)

        for i in tqdm(range(self.num_time_steps + 1), disable=not pgbar):
            if evaluation_time != "inf" and i % evaluation_time == 0 and len(self._results) > 0:
                index = i // evaluation_time
                current_results = self.evaluate_operators()
                print("N :" , current_results[0] , np.abs(current_results[0]))
                self._results[0:-1, index] = current_results
                # Save current time
                self._results[-1, index] = i*self.time_step_size  

            self.run_one_time_step() 

            ########### EXAPNSION ###########
            before_ex_total_bond_ttn = self.state.total_bond_dim()
            
            if (i+1) % (self.expansion_steps+1) == 0 and should_expand:  
                ######## T3NS ########
                ttn = copy.deepcopy(self.state)
                t3n , _  = max_two_neighbour_form(ttn , self.t3n_dict)
                
                adjust_operator_to_ket(t3no,t3n)
                adjust_bra_to_ket(t3n)
                before_ex_total_bond_t3ns = t3n.total_bond_dim()

                self.SVDParameters = replace(self.SVDParameters, max_bond_dim = t3n.max_bond_dim())
                print("SVD MAX :" , self.SVDParameters.max_bond_dim)

                print("tol :" , tol)       
                state_ex_t3n = expand_subspace(t3n, 
                                            t3no, 
                                            self.num_vecs, 
                                            self.tau, 
                                            self.SVDParameters, 
                                            tol, 
                                            self.Lanczos_threshold, 
                                            self.k_fraction, 
                                            self.validity_fraction, 
                                            self.increase_fraction,
                                            self.max_iter,
                                            self.KrylovBasisMode)
                after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                state_ex_ttn = original_form(state_ex_t3n , self.t3n_dict)
                expanded_dim_tot = state_ex_ttn.total_bond_dim() - ttn.total_bond_dim()
                if  expanded_dim_tot > self.rel_tot_bond:
                    print("expanded_dim_tot :" , expanded_dim_tot)
                    A = True
                    # tol_prime = tol
                    for _ in range(10):
                        if A:
                            tol *= self.tol_step
                            print("1) tol" , tol)                            
                            state_ex_t3n_prime = expand_subspace(t3n, 
                                                                t3no, 
                                                                self.num_vecs, 
                                                                self.tau, 
                                                                self.SVDParameters, 
                                                                tol, 
                                                                self.Lanczos_threshold, 
                                                                self.k_fraction, 
                                                                self.validity_fraction, 
                                                                self.increase_fraction,
                                                                self.max_iter,
                                                                self.KrylovBasisMode)
                            after_ex_total_bond_t3ns = state_ex_t3n_prime.total_bond_dim()
                            state_ex_ttn = original_form(state_ex_t3n_prime , self.t3n_dict)
                            expanded_dim_tot = state_ex_ttn.total_bond_dim() - ttn.total_bond_dim()
                            print("2) expanded_dim :" , expanded_dim_tot)
                            if expanded_dim_tot < 0:
                                state_ex_ttn = ttn
                                tol /= self.tol_step
                                A = False
                            elif expanded_dim_tot < self.rel_tot_bond :  
                                A = False  
                self.state = normalize_ttn_Lindblad_1_conj(self.state)
                self.adjust_to_initial_structure()                 
                #self._orthogonalize_init(force_new=True)                     

                if self.max_bond < state_ex_ttn.total_bond_dim():
                    print(self.max_bond , state_ex_ttn.total_bond_dim()) 
                    state_ex_ttn = ttn
                    should_expand = False
                    print("3")

                if state_ex_ttn.total_bond_dim() - before_ex_total_bond_ttn <= 0:
                   state_ex_ttn = ttn
                   tol /= self.tol_step
                   print(state_ex_ttn.total_bond_dim() , before_ex_total_bond_ttn)
                   print("4")      

                self.state = state_ex_ttn
                after_ex_total_bond_ttn = self.state.total_bond_dim()

                expanded_dim_total_bond_ttn = after_ex_total_bond_ttn - before_ex_total_bond_ttn
                expanded_dim_total_bond_t3ns = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns

                if self.max_bond < after_ex_total_bond_ttn:
                    print("END :" , after_ex_total_bond_ttn)
                    should_expand = False      
                
                print("expanded_dim T3NS:" , expanded_dim_total_bond_t3ns)      
                print("T3NS:" , before_ex_total_bond_t3ns , "--->" , after_ex_total_bond_t3ns)   

                print("expanded_dim TTN:" , expanded_dim_total_bond_ttn)
                print("TTN:" , before_ex_total_bond_ttn , "--->" , after_ex_total_bond_ttn)    
 
            ##################################
                 
            self.record_bond_dimensions()
                    
        if evaluation_time == "inf":
            current_results = self.evaluate_operators()
            self._results[0:-1, 0] = current_results
            self._results[-1, 0] = i*self.time_step_size
        if filepath != "":
            self.save_results_to_file(filepath)         



    # RUN entirly in T3NS form                
    def run_ex_t3n(self, evaluation_time: Union[int,"inf"] = 1, filepath: str = "",
            pgbar: bool = True,):
        """
        Runs this time evolution algorithm for the given parameters.

        The desired operator expectation values are evaluated and saved.

        Args:
            evaluation_time (int, optional): The difference in time steps after which
                to evaluate the operator expectation values, e.g. for a value of 10
                the operators are evaluated at time steps 0,10,20,... If it is set to
                "inf", the operators are only evaluated at the end of the time.
                Defaults to 1.
            filepath (str, optional): If results are to be saved in an external file,
                the path to that file can be specified here. Defaults to "".
            pgbar (bool, optional): Toggles the progress bar. Defaults to True.
        """
        self._init_two_neighbour_form() 
        should_expand = True
        self._init_results(evaluation_time)
        assert self._results is not None
        tol = self.initial_tol
        I = TTNO.Identity(self.state)

        for i in tqdm(range(self.num_time_steps + 1), disable=not pgbar):
            t3n_copy_1 = copy.deepcopy(self.state)

            if  self.normalize:
                #orth_center_id_1 = self.state.root_id
                #orth_center_id_2 = orth_center_id_1.replace('Site', 'Node')
                #self.state = normalize_ttn_Lindblad_3_conj(t3n_copy_1 , orth_center_id_1 , orth_center_id_2) # better than 1 and 4
                #self._orthogonalize_init(force_new=True)

                update_path_0 = self.update_path[0]
                self.state = normalize_ttn_Lindblad_4(t3n_copy_1 , update_path_0) 
                
                self.partial_tree_cache = PartialTreeCachDict()
                self._init_partial_tree_cache()                 
                norm = self.state.operator_expectation_value_Lindblad(I)
                #print("Norm :" ,norm, np.abs(norm))
            else:
                self.state = t3n_copy_1  
                norm = t3n_copy_1.operator_expectation_value_Lindblad(I)
                #print("Norm :" ,norm, np.abs(norm))

            if evaluation_time != "inf" and i % evaluation_time == 0 and len(self._results) > 0:
                index = i // evaluation_time
                current_results = self.evaluate_operators()
                #print("H :" , self.evaluate_operators()[0] / norm)
                self._results[0:-1, index] = current_results
                # Save current time
                self._results[-1, index] = i*self.time_step_size  

            self.run_one_time_step_ex() 

            ########### EXAPNSION ###########
            before_ex_total_bond_t3ns = self.state.total_bond_dim()
            
            if (i+1) % (self.expansion_steps+1) == 0 and should_expand:  
                
                ######## T3NS ########
                t3n = copy.deepcopy(self.state)
                t3no = copy.deepcopy(self.hamiltonian)
                        
                adjust_operator_to_ket(t3no,t3n)
                adjust_bra_to_ket(t3n)
                print("tol :" , tol)   
                self.SVDParameters = replace(self.SVDParameters, max_bond_dim = t3n.max_bond_dim())
                print("max_bond_dim :" , self.SVDParameters.max_bond_dim)
                state_ex_t3n = expand_subspace(t3n, 
                                            t3no, 
                                            self.num_vecs, 
                                            self.tau, 
                                            self.SVDParameters, 
                                            tol, 
                                            self.Lanczos_threshold, 
                                            self.k_fraction, 
                                            self.validity_fraction, 
                                            self.increase_fraction,
                                            self.max_iter,
                                            self.KrylovBasisMode)
                after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                expanded_dim_tot = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns
                if expanded_dim_tot <= 0:
                    state_ex_t3n = t3n
                    tol /= self.tol_step

                if  expanded_dim_tot > self.rel_tot_bond:
                    print("expanded_dim_tot :" , expanded_dim_tot)
                    A = True
                    # tol_prime = tol
                    for _ in range(10):
                        if A:
                            tol *= self.tol_step
                            print("1) tol" , tol)                            
                            state_ex_t3n = expand_subspace(t3n, 
                                                                t3no, 
                                                                self.num_vecs, 
                                                                self.tau, 
                                                                self.SVDParameters, 
                                                                tol, 
                                                                self.Lanczos_threshold, 
                                                                self.k_fraction, 
                                                                self.validity_fraction, 
                                                                self.increase_fraction,
                                                                self.max_iter,
                                                                self.KrylovBasisMode)
                            after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                            expanded_dim_tot = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns

                            print("2) expanded_dim :" , expanded_dim_tot)
                            if expanded_dim_tot <= 0:
                                state_ex_t3n = t3n
                                tol /= self.tol_step
                                A = False
                            elif expanded_dim_tot < self.rel_tot_bond :  
                                A = False                

                if self.max_bond < state_ex_t3n.total_bond_dim():
                    print(self.max_bond , state_ex_t3n.total_bond_dim()) 
                    state_ex_t3n = t3n
                    should_expand = False
                    print("3") 

                self.state = state_ex_t3n
                self._orthogonalize_init(force_new=True)

                after_ex_total_bond_t3ns = state_ex_t3n.total_bond_dim()
                expanded_dim_tot = after_ex_total_bond_t3ns - before_ex_total_bond_t3ns 
                                   
                print("expanded_dim T3NS:" , expanded_dim_tot)      
                print("T3NS:" , before_ex_total_bond_t3ns , "--->" , after_ex_total_bond_t3ns)      
 
            ##################################
                 
            self.record_bond_dimensions()
                    
        if evaluation_time == "inf":
            current_results = self.evaluate_operators()
            self._results[0:-1, 0] = current_results
            self._results[-1, 0] = i*self.time_step_size
        if filepath != "":
            self.save_results_to_file(filepath)   


    
